In [1]:
import sys
sys.path.insert(0, "..")

In [2]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [3]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

# Data Processing
In this notebook, we will be processing the data in the ACM Dataset.

The first step is to understand the information we want to extract from our dataset.

In [4]:
PATH="../data/ACM Data/"

In [5]:
import glob
files = glob.glob(f'{PATH}/*.xml')
files[:5]

['../data/ACM Data/PROC-SWIM12-2012-2237867.xml',
 '../data/ACM Data/PROC-VALUETOOLS09-2009-1698822.xml',
 '../data/ACM Data/PROC-MODELS06-2006-1762828.xml',
 '../data/ACM Data/PROC-HRI12-2012-2157689.xml',
 '../data/ACM Data/PROC-SIGGRAPHASIA09-2009-1666778.xml']

In [6]:
from urop.xml import xml_files_to_elements, elements_to_cleaned_text
import itertools
elements = xml_files_to_elements(files)
clean_text = elements_to_cleaned_text(elements)

print(list(itertools.islice(files, 1)))

['../data/ACM Data/PROC-SWIM12-2012-2237867.xml']


In [8]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Error loading stopwords: <urlopen error [Errno 111]
[nltk_data]     Connection refused>
[nltk_data] Error loading wordnet: <urlopen error [Errno 111]
[nltk_data]     Connection refused>
[nltk_data] Error loading punkt: <urlopen error [Errno 111] Connection
[nltk_data]     refused>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [Errno 111] Connection refused>


False

Signature: nltk.download(info_or_id=None, download_dir=None, quiet=False, force=False, prefix='[nltk_data] ', halt_on_error=True, raise_on_error=False)
Source:   
    def download(self, info_or_id=None, download_dir=None, quiet=False,
                 force=False, prefix='[nltk_data] ', halt_on_error=True,
                 raise_on_error=False):
        # If no info or id is given, then use the interactive shell.
        if info_or_id is None:
            # [xx] hmm -- changing self._download_dir here seems like
            # the wrong thing to do.  Maybe the _interactive_download
            # function should make a new copy of self to use?
            if download_dir is not None: self._download_dir = download_dir
            self._interactive_download()
            return True

        else:
            # Define a helper function for displaying output:
            def show(s, prefix2=''):
                print(textwrap.fill(s, initial_indent=prefix+prefix2,
                          

In [ ]:
from urop.nlp import clean_text

In [ ]:
df['cleaned_text'] = df['fulltext'].apply(clean_text)

In [ ]:
df.sample(10)

In [ ]:
df.iloc[42]["cleaned_text"]

We notice that some items don't have text at all, and we ignore them, along with any documents with less than 150 characters.

In [ ]:
mask = (df['cleaned_text'].str.len() > 150)
df = df[mask]

In [ ]:
df.info()

In [ ]:
df.sample(10)

In [37]:
import itertools
from urop.xml import ACMCorpus
from gensim import corpora

corpus = ACMCorpus(PATH)
# dictionary = corpora.Dictionary(corpus)
for text in corpus:
    print(text[0:10])

2018-03-06 02:50:35,722 : INFO : Parsing ../data/ACM Data/PROC-SWIM12-2012-2237867.xml
2018-03-06 02:50:35,754 : INFO : Parsing ../data/ACM Data/PROC-VALUETOOLS09-2009-1698822.xml
2018-03-06 02:50:35,934 : INFO : Parsing ../data/ACM Data/PROC-MODELS06-2006-1762828.xml


Automatic 
Towards Be
Payasyougo
Named Enti
Permission
Type Infer
A decentra
SPOVC A sc
SPARQL que
Abstract F
FlowMonito
Spectrumaw
Design and
rfactorial

WiMAX Simu
A Realisti
A simulati
Predictive
Automated 
RealTime P
Dynamic Tr
Incorporat
A Software
A timeawar
Approximat
On MntMntS
Transient 
Computatio
Structured
Memory Efc
Analysis o
Using Bulk
Heavy traf
Reliable a
Finding St
Community 
A powerlaw
Parallelis
Simulation
A Performa
Asymptotic
Population
A random w
COINC Libr
EndtoEnd D
NetworkCal
New Stocha
A Mean Fie
Throughput
Measuring 
Hybrid Sim
From parti
Interfaces
Regenerati
A Practica
Enhanced I
Estimating
Response t
Complex Sy
Spatial fa
Analytical
Stability 
Utility Op
A Scaling 
A Recursiv
A Game The
An informa
The Concer
A Dynamic 
Stochastic
Myopic ver
Equilibriu
Equilibriu
Jamming ga
Coalitiona
GameTheore
Resource A
A Game The
Linear and
On the Bas
A Potentia
Multiple a
From Rosen
The Impact
Feasibilit
Towards Co
Markov Cha


2018-03-06 02:50:35,947 : INFO : Parsing ../data/ACM Data/PROC-HRI12-2012-2157689.xml












































Strategies
Grip Force
Designing 
Robot Beha
Do People 
Social Fac
New Measur
Consistenc
Real World
Effects of
Teamwork i
Towards Hu
Designing 
A Touchscr
Children s
HumanRobot
Dynamic Ge
Design of 
User Exper
Visual Cue
Socially C
Sensorless
Assitive T
If You Sou
Beyond Spa
Attentiona
Midas Touc
Facial Ges
Design Int
Handheld O
Unveiling 
Assessing 
Does a Rob
Towards A 
Using the 
Attracting
An Intenti
Gestonurse
Manipulati
Studying V
A FollowUp
Personalit
Understand
Cultural S
Robot Comp
Applying T
Deep Netwo
User Atten
Tracking A
Real Time 
Integratin
HumanRobot
VisionBase
A Prototyp
Explaining
Applying P
Transfer f
A Probabil
Control of
Referent I
Listener A
Can You Ho
Captain Ma
Online Gam
The Effect
Mechanical
Shared Gaz
A Social R
Modied Soc
The Roomba
How to Use
Ask Inform
Creating H
Unsupervis
Immersive 
Don t Stan
Coupled In
Developing
Is the Soc
AnimalInsp
Spatial La
Potential 
Affect Mis
Policy Tra
Exploratio
Dimensions
Extending 
L

2018-03-06 02:50:36,172 : INFO : Parsing ../data/ACM Data/PROC-SIGGRAPHASIA09-2009-1666778.xml
2018-03-06 02:50:36,206 : INFO : Parsing ../data/ACM Data/PROC-INFOSECCD09-2009-1940976.xml
2018-03-06 02:50:36,254 : INFO : Parsing ../data/ACM Data/PROC-SE4SG2014-2014-2593845.xml
2018-03-06 02:50:36,256 : INFO : Parsing ../data/ACM Data/PROC-DOCENG08-2008-1410140.xml
2018-03-06 02:50:36,352 : INFO : Parsing ../data/ACM Data/PROC-COP11-2011-2068736.xml


...

2018-03-06 02:51:32,875 : INFO : Parsing ../data/ACM Data/PROC-AFRIGRAPH09-2009-1503454.xml
2018-03-06 02:51:32,906 : INFO : Parsing ../data/ACM Data/PROC-AGENTS97-1997-267658.xml
2018-03-06 02:51:33,003 : INFO : Parsing ../data/ACM Data/PROC--1980-800227.xml



Architectu
Capacity o
Joint Chan
Characteri
MA P Media
Effects of
On Designi
Challenges
Challenges
Challenge 
Design and
Model T An
SelfManage
Modeling M
An Analyti
MoB A Mobi
PeopleNet 
Experiment
Using Mobi
Barrier Co
CrossLayer
Virtual Re
Revisiting
PickbyVisi
Associatin

Simulating
Virtual Re
Particleba
Visual qua
Using An I
An Efcient
Programmab



































































































2018-03-06 02:51:33,071 : INFO : Parsing ../data/ACM Data/PROC-CVM12-2012-2428303.xml
2018-03-06 02:51:33,085 : INFO : Parsing ../data/ACM Data/PROC-CHI82-1982-800049.xml
2018-03-06 02:51:33,196 : INFO : Parsing ../data/ACM Data/PROC-SPAA14-2014-2612669.xml


AN OVERVIE
TUTORIAL O
Abstractio
TUTORIAL O
DATA ABSTR
TUTORIAL O
What Shoul
Rowe Data 
BEHAVlOUR 
Presentati
CONSISTENC
Relationsh
APPLICATIO
Workshop S
AN INTERAC
INTEGRATIO
DYNAMIC SY
Data and A
Data Abstr
The Use of
Default Re
Integratin
DATA MODEL
ON THE SPE
CONSTRAINT
Some Contr
Integratin
PROCESSING
A LOGICAL 
MEDIATING 
DATA DICTI
Heterogene
Modelling 
RELATIVISM
ASSOCIATIN
A Data Abs
Incomplete
AXIOMATIC 
Use of Dat
Make More 
On Concept
A THREELEV
A PERSPECT
Abstract D
DATA BASES
MULTIPLE P
ISSUES IN 
TOWARDS CO
DATA ABSTR
Abstractio
DATABASE N
A Conceptu
POSITION P
The Extens
ARE DATA T
Constructi
Types Alge

































LEARNING A
Learning a
Evaluating
COMPUTER C
PSYCHOLOGI
Aaron Marc
A SYSTEMS 
THE DESIGN
WINDOWING 
NOTETAKING
TAPPING IN
HANCOMPUTE
Heuristics
POLITICAL 
HOW ACCEPT
HUMAN RELA
SOFTWARE G
A TestBed 
CONTROVERS
DMS A COMP
Comparison
TOWARD THE
INDENTATIO
AN EMPIRIC
A Theoreti
THE IMPACT
Evaluation
An Ease of
AN ANALYSI
CAN WE EXP
AN AUTOMAT

KeyboardInterrupt: 